In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install easyocr

In [ ]:
DATASET_FOLDER = r'/content/drive/MyDrive/student_resource_3/dataset/'

In [ ]:
import easyocr
import re
import cv2
import numpy as np
import pandas as pd
import os
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import time
from PIL import Image
# Initialize EasyOCR Reader
reader = easyocr.Reader(['en'],gpu=True)

train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))
#regex
weight_pattern =   r'\b\d+(\.\d+)?\s*(fl oz|oz|g|kg|grams?|kilograms?|kilogram|carat|ounce|pound|lb|mg|lbs|microgram|ton)\b'
volume_pattern =   r'\b(\d+\.\d+)?\s*?(cup|millilitre|milliliter|litre|liter|decilitre|deciliter|pint|quart|gallon|centilitre|centiliter|ml|l|dl|gal|cl|cml)\b'
length_pattern =   r'\b\d+(\.\d+)?\s*?(inch|foot|feet|metre|meter|centimetre|centimeter|millimetre|millimeter|cm|m|mm|in|ft|yd|yard)\b'
electricity_pattern =   r'\b(\d+(\.\d+)?)\s*?(watt|kilowatt|kilowatts|horsepower|gigabyte?|volt|w|kw|hp|gb|v|bhp|V)\b'
light_pattern =  r'\b(\d+\.\d+)?\s*?(candela|nits|cd|nit)\b'
other_pattern =   r'\b\d+(\.\d+)?\s*?(person|persons|hour|hr|min|mins|sec|secs)\b'
#functions to check regex
def check_length(text):
    return re.finditer(length_pattern, text.lower())

def check_volume(text):
    return re.finditer(volume_pattern, text.lower())

def check_electricity(text):
    return re.finditer(electricity_pattern, text.lower())

def check_light(text):
    return re.finditer(light_pattern, text.lower())

def check_other(text):
    return re.finditer(other_pattern, text.lower())

def check_weight(text):
    return re.finditer(weight_pattern, text.lower())
multiple_matches= []
def process_row(index,row):
    prediction = ""

    image_url = row['image_link']
    print(index)
    # Fetch and process the image
    response = requests.get(image_url)
    if response.status_code == 200:
        # Fetch image directly into a numpy array
        image_np = np.frombuffer(response.content, np.uint8)
        image_cv2 = cv2.imdecode(image_np, cv2.IMREAD_COLOR)

        # Resize image early to a smaller size for faster processing
        height, width = image_cv2.shape[:2]
        new_width = 800  # Set desired width
        new_height = int((new_width / width) * height)
        image_resized = cv2.resize(image_cv2, (new_width, new_height), interpolation=cv2.INTER_AREA)

        # Perform OCR with EasyOCR on the resized image
        results = reader.readtext(image_resized)
        text = ' '.join([result[1] for result in results])
        print(text)
        # Determine prediction based on entity_name
        if row['entity_name'] == 'item_volume':
            prediction = check_volume(text)
        elif row['entity_name'] in ['height', 'width', 'depth']:
            prediction = check_length(text)
        elif row['entity_name'] == 'voltage' or row['entity_name'] == 'wattage':
            prediction = check_electricity(text)
        elif row['entity_name'] in ['item_weight', 'maximum_weight_recommendation']:
            prediction = check_weight(text)
        elif row['entity_name'] == 'other':
            prediction = check_other(text)
        else:
            print("Entity name not recognized")
 #concatenate matches
    concatenated_matches = ', '.join([f"{match[0]}" for match in prediction])
    multiple_matches.append(index)
    row_df = pd.DataFrame({'index': [index], 'prediction': [concatenated_matches]})
    return row_df

if __name__ == '__main__':
    test_out = pd.DataFrame(columns=['index', 'prediction'])
    start = time.time()
    test_from6361 = test.iloc
    row_data = list(test_from6361.iterrows())
    # Convert the results back to a DataFrame
    for index,row in row_data:
        results = process_row(index,row)
        # print(results)
        test_out = pd.concat([test_out, results], ignore_index=True)
        test_out.to_csv("from_131187.csv", index=False)
    end_time = time.time()
    print(f"Time taken: {end_time - start:.2f} seconds")


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

24998
8 3 L 26 39 cm cm C.
24999
5 31 5 7 31.75cm/12.5in 7 Sin 75cm/12.
25000
12CM 38CM 9.5CM
Time taken: 1.03 seconds
